In [1]:
import json
import os
import pickle
from tqdm import tqdm
import numpy as np

In [2]:
def ospif(file):
    return os.path.isfile(file)

def ospid(dir_):
    return os.path.isdir(dir_)

def pkl_dmp(obj, fp):
    with open(fp, "wb") as fo:
        pickle.dump(obj, fo, protocol=pickle.HIGHEST_PROTOCOL)
        
def pkl_ld(fp):
    with open(fp, "rb") as fi:
        pkl_content = pickle.load(fi)
    return pkl_content

def json_ld(fp):
    with open(fp, "r") as fi:
        json_content = json.load(fi)
    return json_content

def json_dmp(obj, fp, indent=None):
    with open(fp, "w") as fo:
        if indent is None:
            json.dump(obj, fo)
        else:
            assert isinstance(indent, int)
            json.dump(obj, fo, indent=indent)

In [3]:
DATASET_FP = f"../../data/ego_exo4d/labels/test.pkl"
assert ospif(DATASET_FP), print(DATASET_FP)

RESULTS_ROOT_DR = f"../../runs"
assert ospid(RESULTS_ROOT_DR)

RESULTS_SUBDIR = "egoExo4d_release" 

RESULTS_FP = f"{RESULTS_ROOT_DR}/{RESULTS_SUBDIR}/test_index2logits_checkpoint-maxCaptioningScore.json"
assert ospif(RESULTS_FP)

DUMP_FP = f"{RESULTS_ROOT_DR}/{RESULTS_SUBDIR}/take2startNendTimestamp2predScores_checkpoint-maxCaptioniningScore.pkl" 

In [4]:
dct_rslts =json_ld(RESULTS_FP)
dct_dtst = pkl_ld(DATASET_FP)

In [5]:
cnt = 0
for k, v in dct_dtst.items():
    cnt += len(v)
assert cnt == len(dct_rslts)

In [6]:
dmp_dct = {}
idx = 0
for k, v in dct_dtst.items():
    assert k not in dmp_dct
    dmp_dct[k] = {}
    for k1 in v:
        strt_tmstmp = k1[1]
        end_tmstmp = k1[2]
        assert str(idx) in dct_rslts

        # print((strt_tmstmp, end_tmstmp))
        assert (strt_tmstmp, end_tmstmp) not in dmp_dct[k]
        assert isinstance(dct_rslts[str(idx)], list)
        assert isinstance(dct_rslts[str(idx)][0], float)
        dmp_dct[k][(strt_tmstmp, end_tmstmp)] = dct_rslts[str(idx)]
        
        idx += 1
        # break
    # break

In [7]:
pkl_dmp(dmp_dct, DUMP_FP)